### Tarea 7.9

Use un polinomio interpolante para aproximar la integral de una gaussiana con $\sigma = 1$ y $\mu = 0$, entre $x = 0$ y $x = 1$. Compare con el resultado obtenido a partir de

$$
\int_0^a dx\,e^{-x^2} = \frac{\sqrt{\pi}}{2}\text{erf}(a)
$$

donde $\text{erf}(a)$ es la función de error y en python se puede llamar usando scipy.special.erf del modulo scipy.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import special

Vamos a usar los polinomios de Legendre, que son de la forma:

$$
P(x) = a_{0} + a_{1}(x - x_{0}) + a_{2}(x - x_{0})(x - x_{1}) ... + a_{n}(x - x_{0})...(x - x_{n - 1})
$$


Para ello usaremos 3 puntos, entonces el polinomio quedaría de la forma:

$$
P(x) = a_{0} + a_{1}(x - x_{0}) + a_{2}(x - x_{0})(x - x_{1})
$$

Entonces la integral de este polinomio es:

$$
\int_{a}^{b} dx P(x) = \int_{a}^{b} dx (a_{0} + a_{1}(x - x_{0}) + a_{2}(x - x_{0})(x - x_{1}))
$$


La integral sería:

$$
\int_{a}^{b} dx P(x) = x \left[a_{0} + \frac{a_{1}x}{2} - a_{1}x_{0} + \frac{a_{2}x^{2}}{3} - \frac{a_{2}x x_{1}}{2} - \frac{a_{2}x_{0}x}{2} + a_{2}x_{0}x_{1}\right]^{b}_{a}
$$

Esto nos servirá para calcular la integral aproximada de la función que queremos.
Ahora queremos la lista de coeficientes del polinomio. Hay un total de 3 coeficientes en este caso.

In [2]:
def diferencias_divididas(puntos):
    '''
    Esta función obtiene los coeficientes del polinomio de
    Legendre en función de los puntos del mismo.
    
    Input:

    puntos = son los puntos del polinomio.
    '''
    n = len(puntos) - 1 # Número del coeficientes
    Fs_coef = [np.zeros(n + 1)] # Son los coeficientes de del polinomio.
    for i in range(n + 1):
        Fs_coef[0][i] = puntos[i][1] # El coeficiente 0 del polinomio equivale 
                                    # al punto número 1
    for i in range(1, n + 1):
        Fs_coef.append(np.zeros(n + 1 - i))
        for j in range(1, i + 1):
            Fs_coef[j][i-j] = (Fs_coef[j-1][i-j+1] - Fs_coef[j-1][i-j])/(puntos[i][0] - puntos[i-j][0])
    
    return [Fs_coef[i][0] for i in range(n + 1)]
    # Se agregan los coeficientes mediante la compresión de listas segun el grado
    # del polinomio.

In [3]:
def Integración_Legendre(coeficientes, puntos, limite_de_integracion = [0,1]):
    
    '''
    Esta función calcula la integral del polinomio de Legendre.
    
    Inputs:
    
    coeficientes = son los coeficientes del polinomio.
    
    puntos = los puntos del polinomio. En este caso vamos a usar 3
    
    limite_de_integración = el intervalo de evaluación de la integral.
    '''
    a_0,a_1,a_2 = coeficientes
    x_0,x_1,x_2 = puntos[:,0] # Son los puntos iniciales.
    a,b = limite_de_integracion
    
    Integral_2 = b*(a_0 + (a_1*b)/2 - a_1*x_0 + (a_2*b**(2))/3 - (a_2*b*x_1)/2 - (a_2*x_0*b)/2 + a_2*x_0*x_1)
    # Es la integral definida dentro de [a,b] en b.
    
    Integral_1 = a*(a_0 + (a_1*a)/2 - a_1*x_0 + (a_2*a**(2))/3 - (a_2*a*x_1)/2 - (a_2*x_0*a)/2 + a_2*x_0*x_1)
    # Es la integral definida en [a,b] en a.
    return Integral_2 - Integral_1
    # Se procede a usar la definición de Integral definida.

In [4]:
x = np.linspace(0,1,3)
puntos = np.array([[xi, np.exp((-1)*xi**2)] for xi in x])
# Acá lo que se hace es la compresión de listas como un arreglo de los puntos
# de la función gaussiana. En este caso seran 3

coeficientes = diferencias_divididas(puntos) # Como ya definimos, se hace la diferencias
                                       # divididas para calcular los coeficientes del
                                       # polinomio en funcion del número de puntos.

Integración_Polinomio_Legendre = Integración_Legendre(coeficientes, puntos, (0,1)) # Se calcula la integral aproximada
                                                                                  # con el polinomio de Legendre
                     
Integral_Gaussiana = (np.sqrt(np.pi)/2)*special.erf(1) # Calculamos la gaussiana con la formula
                                                       # definida anteriormente.

print("La integral de legendre aproximada a la gaussiana es",Integración_Polinomio_Legendre  )
print("La integral de la gaussiana es", Integral_Gaussiana) 


La integral de legendre aproximada a la gaussiana es 0.7471804289095103
La integral de la gaussiana es 0.7468241328124269


Para caluclar el error, definimos:

In [5]:
Error= (abs(Integración_Polinomio_Legendre - Integral_Gaussiana)/abs(Integral_Gaussiana))*100 
# Este es el error porcentual.
                                                                                              
print("La integral se calculó con un error del", Error, "%")

La integral se calculó con un error del 0.04770816600980385 %


No se aleja mucho de la integral real.

# Calificación: 7.0